### Deep FBA Trader model creation

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, InputLayer
from tensorflow.keras.utils import Sequence
import numpy as np
import matplotlib.pyplot as plt
import joblib
import os

Combining training data into one csv file

In [3]:
# Folder paths
root_folder_path = 'C:/Users/camer/Documents/Masters Thesis/Data/Training data/'
specific_folder = '1 sec batch full trading day'
folder_path = os.path.join(root_folder_path, specific_folder)

# Load the scaler
scaler_filename = 'scalers/scaler_equil_predictor.joblib'
scaler = joblib.load(scaler_filename)

In [8]:
model = Sequential()
model.add(InputLayer(input_shape=(1, len(pd.read_csv(os.path.join(folder_path, os.listdir(folder_path)[0]), nrows=1).columns) - 2), dtype=tf.float16))
model.add(LSTM(10, activation='relu'))
model.add(Dense(5, activation='relu'))
model.add(Dense(3, activation='relu'))
model.add(Dense(1))

adam_optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=adam_optimizer, loss='mean_squared_error')

# checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
#     filepath='Neural_network_models/equil_pred_model/epoch_{epoch:02d}.keras',
#     save_freq='epoch',
#     save_best_only=False,
#     verbose=1
# )

# history = model.fit(train_generator, epochs=20, validation_data=test_generator, verbose=1)

C:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\core\input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [ ]:
for i in range(10):
    for filename in os.listdir(folder_path):
        if filename.endswith(".csv"):
            file_path = os.path.join(folder_path, filename)
            print(f"Training on {filename}")

            data = pd.read_csv(file_path)

            X = data.drop(columns=['time_of_trade', 'final_trade_price']).values
            y = data['final_trade_price'].values

            X_scaled = scaler.transform(X)

            X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))

            X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

            model.fit(X_train, y_train, epochs=1, validation_data=(X_val, y_val), verbose=1)

Training on training_data_20240802_192647.csv
36082/36082 ━━━━━━━━━━━━━━━━━━━━ 193s 5ms/step - loss: 465.5968 - val_loss: 361.6128
Training on training_data_20240802_192651.csv
36136/36136 ━━━━━━━━━━━━━━━━━━━━ 187s 5ms/step - loss: 349.4220 - val_loss: 344.4655
Training on training_data_20240802_192654.csv
36481/36481 ━━━━━━━━━━━━━━━━━━━━ 194s 5ms/step - loss: 344.6210 - val_loss: 344.0499
Training on training_data_20240802_192657.csv
36288/36288 ━━━━━━━━━━━━━━━━━━━━ 193s 5ms/step - loss: 339.9725 - val_loss: 336.7047
Training on training_data_20240802_192701.csv
36178/36178 ━━━━━━━━━━━━━━━━━━━━ 180s 5ms/step - loss: 348.9612 - val_loss: 345.2880
Training on training_data_20240802_192704.csv
36118/36118 ━━━━━━━━━━━━━━━━━━━━ 182s 5ms/step - loss: 360.7718 - val_loss: 365.3134
Training on training_data_20240802_192708.csv
36146/36146 ━━━━━━━━━━━━━━━━━━━━ 190s 5ms/step - loss: 350.1696 - val_loss: 346.3979
Training on training_data_20240802_192713.csv
35992/35992 ━━━━━━━━━━━━━━━━━━━━ 183s

46678/46678 ━━━━━━━━━━━━━━━━━━━━ 224s 5ms/step - loss: 332.4346 - val_loss: 331.0124
Training on training_data_20240803_093722.csv
46721/46721 ━━━━━━━━━━━━━━━━━━━━ 235s 5ms/step - loss: 327.8800 - val_loss: 327.0521
Training on training_data_20240803_093727.csv
46764/46764 ━━━━━━━━━━━━━━━━━━━━ 236s 5ms/step - loss: 311.5764 - val_loss: 311.1535
Training on training_data_20240803_093730.csv
46679/46679 ━━━━━━━━━━━━━━━━━━━━ 231s 5ms/step - loss: 334.8799 - val_loss: 334.4203
Training on training_data_20240803_093733.csv
32735/47302 ━━━━━━━━━━━━━━━━━━━━ 1:01 4ms/step - loss: 320.4251

In [ ]:
model.save('Neural_network_models/1secbatch_model_v6_current_best.keras')
joblib.dump(history.history, 'training_history_equil_pred_v6.pkl')

In [8]:
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

NameError: name 'history' is not defined

<Figure size 1000x600 with 0 Axes>

Prepping the training data for input - No longer in use

In [4]:
X = training_data.drop(columns=['final_trade_price', 'time_of_trade'])
y = training_data['final_trade_price']

X = X.values
y = y.values

X = X.reshape((X.shape[0], 1, X.shape[1]))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_2d = X_train.reshape((X_train.shape[0], X_train.shape[2]))
X_test_2d = X_test.reshape((X_test.shape[0], X_test.shape[2]))

X_train_scaled = scaler.fit_transform(X_train_2d)
X_test_scaled = scaler.transform(X_test_2d)

scaler_filename = 'scaler.joblib'
joblib.dump(scaler, scaler_filename)

X_train_scaled = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_scaled = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

X_train_scaled = X_train_scaled.astype(np.float16)
X_test_scaled = X_test_scaled.astype(np.float16)

Building the model

In [ ]:
model = Sequential()
model.add(InputLayer(input_shape=(X_train_scaled.shape[1], X_train_scaled.shape[2]), dtype=tf.float16))
model.add(LSTM(10, activation='relu'))
model.add(Dense(5, activation='relu'))
model.add(Dense(3, activation='relu'))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')

history = model.fit(X_train_scaled, y_train, epochs=20, validation_split=0.2, verbose=1)
test_loss = model.evaluate(X_test_scaled, y_test, verbose=1)

Save model

In [ ]:
model.save('Neural_network_models/1secbatch_model_v6_current_best.keras')

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()